In [ ]:
TRAIN_DATA = "DATA/train.csv"
TEST_DATA = "DATA/test.csv"
SUBMISSON_DATA = "DATA/submission.csv"

In [ ]:
import pandas as pd
import statsmodels.api as sm

In [ ]:
df = pd.read_csv(TRAIN_DATA)
df

In [ ]:
df.dropna(subset=['Содержание'], inplace=True)

In [ ]:
df[df['Содержание'].str.contains('тест', case=False)]

In [ ]:
tdf = df[df['Статус'] == 'Отменен']
tdf['Тип переклассификации'].value_counts()

In [ ]:
priority = {'3-Низкий': 3, '2-Средний': 2, '1-Высокий': 1, '0-Критический': 0}
status = {'Отменен': 0, 'Закрыт': 1}
obr = {'Запрос': 0, 'Инцидент': 1}
critical = {'4-Нет': 4, "3-Базовая": 3, "2-Повышенная": 2, "1-Особая": 1}
influence = {"4-Нет влияния": 4, "3-Малое": 3, "2-Значительное": 2, "1-Всеохватывающее": 1}

In [ ]:
df['Приоритет'] = df['Приоритет'].map(priority)
df

In [ ]:
df['Статус'] = df['Статус'].map(status)
df

In [ ]:
df['Тип обращения на момент подачи'] = df['Тип обращения на момент подачи'].map(obr)
df['Тип обращения итоговый'] = df['Тип обращения итоговый'].map(obr)
df

In [ ]:
df['Критичность'] = df['Критичность'].map(critical)
df['Влияние'] = df['Влияние'].map(influence)
df

In [ ]:
# df[df['Дата восстановления'].isna()] drop
df = df.dropna(subset=['Дата восстановления'])
df

In [ ]:
df = df.drop_duplicates(subset=['Дата закрытия обращения', 'Система', 'Место', 'Сервис'])

In [ ]:
df[df['Статус'] != 0]['Тип переклассификации'].value_counts()

In [ ]:
df['Тип переклассификации'].value_counts()

In [ ]:
target = df['Тип обращения итоговый']
features = df[['Приоритет', 'Тип обращения на момент подачи', 'Критичность', 'Статус', 'Влияние']]

In [ ]:
features = sm.add_constant(features)
model = sm.OLS(target, features)
result = model.fit()

In [ ]:
model2 = sm.Logit(target, features)
result2 = model2.fit()
print(result2.summary())

In [ ]:
print(result.summary())

In [ ]:
df.to_csv("DATA/new_train.csv", index=False)